In [18]:
import numpy as np
import pickle 
import matplotlib.pyplot as plt
import data_manager as dm
import sys
%matplotlib inline

In [40]:
with open('C:\\ASM\\DevData\\eating\\meal_detection\\data\\steven_free_data.pkl', 'rb') as file:
    data = pickle.load(file)

In [53]:
def analyze_result_individual_sess(annots, mps, res):    
    if len(mps) != len(res):
        print("Lengths of mps and res are not same")
        sys.exit(0)
    
    mp_count_in_episode    = len(res[res[:, 0]==1, :])
    mp_count_out_episode    = len(res[res[:, 0]==0, :])
    bite_count_in_episode  = len(res[(res[:, 0]==1) & (res[:, 1]==1), :])
    bite_count_out_episode = len(res[(res[:, 0]==0) & (res[:, 1]==1), :])
    
    return [mp_count_in_episode, mp_count_out_episode, bite_count_in_episode, bite_count_out_episode]
    

In [58]:
def analyze_result_all_subjects(win_size, xth, data, mps_all, res_all):
    subject_count = len(data)
    
    output = np.empty((0, 8))
    for subject in range(subject_count):
        for sess in range(len(res_all[subject]["free"])):
            annots = data[subject][sess][1]
            res = res_all[subject]["free"][sess]
            mps = mps_all[3][subject][sess]
            
            r = [win_size, xth, subject, sess]
            rs = analyze_result_individual_sess(annots, mps, res)            
            r.extend(rs)
            r = np.array(r).reshape((1, len(r)))
            output = np.concatenate((output, r))
            
    return output

In [152]:
res = np.empty((0, 8))

for win_size in range(32, 65, 8):
    for xth in range(10, 41, 5):
        folder = 'len_{}_{}_xth_{}_smooth_800'.format(win_size, win_size, xth)        
        path = 'C:\\ASM\\DevData\\eating\\meal_detection\\results\\' + folder +"\\"
        with open(path+"mps.pkl", "rb") as file:
            mps_all = pickle.load(file)
        with open(path+"results\\res_all.pkl", "rb") as file:
            res_all = pickle.load(file)           
        
        #print(folder)
        r = analyze_result_all_subjects(win_size, xth, data, mps_all, res_all)        
        res = np.concatenate((res, r))
        
print(len(res))

875


In [146]:
print(len(res))
#print(res)
#np.savetxt('res.csv', res, delimiter=',')

875


In [147]:
rs = np.empty((0, 6))
for win_size in range(32, 65, 8):        
    for xth in range(10, 41, 5):
        r = res[(res[:,0]==win_size) & (res[:,1]==xth), 4:]                
        r = np.sum(r, axis=0)                
        r = [win_size, xth, r[0], r[1], r[2], r[3]]
        r = np.array(r).reshape((1, len(r)))        
        rs = np.concatenate((rs, r))
#print(rs)

In [ ]:
fig, ax = plt.subplots()
mp_stat = []
for xth in range(10, 41, 5):
    r = rs[rs[:,1]==xth, 2:4]
    r = np.mean(r, axis=0)
    r.astype(int)    
    s = np.sum(r)
    mp_stat.append([-1*xth/10, r[0], r[1], s])


r = mp_stat
r =np.array(r)
#r.astype(int)
#print(r)
plt.plot(r[:, 0], r[:, 1]/1000, label="Inside")
plt.plot(r[:, 0], r[:, 2]/1000, label="Outside")
plt.plot(r[:, 0], r[:, 3]/1000, label="Total")

plt.scatter(r[:, 0], r[:, 1]/1000, color = 'blue')
plt.scatter(r[:, 0], r[:, 2]/1000, color = 'green')
plt.scatter(r[:, 0], r[:, 3]/1000, color = 'red')

plt.xlabel('Accelerometer threshold for X axis')
plt.ylim([0, 45])
plt.xlim([-4, -1])
plt.ylabel('Segment Count (thousands)')
plt.legend(loc=2)

In [ ]:
fig, ax = plt.subplots()

for win_size in range(32, 65, 8):
    mp_stat = []
    for xth in range(10, 41, 5):
        r = rs[(rs[:,0]==win_size) & (rs[:,1]==xth), 4:]                        
        r = np.sum(r, axis=0)
        mp_stat.append([-1*xth/10, r])


    r = mp_stat
    r =np.array(r)   
    plt.plot(r[:, 0], r[:, 1], label="{} s".format(int(win_size/8)))
    plt.plot(r[:, 0], r[:, 1], label="{} s".format(int(win_size/8)))    

    #plt.scatter(r[:, 0], r[:, 1]/1000, color = 'blue')
    #plt.scatter(r[:, 0], r[:, 2]/1000, color = 'green')
    #plt.scatter(r[:, 0], r[:, 3]/1000, color = 'red')

    plt.xlabel('Accelerometer threshold for X axis')
    #plt.ylim([0, 2000])
    #plt.xlim([-4, -1])
    plt.ylabel('Detected Bite Count (thousands)')
    #plt.legend(loc=2)
    
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.25),
          ncol=3, fancybox=True, shadow=True)
plt.show()

In [ ]:
for win_size in range(40, 57, 8):
    fig, ax = plt.subplots()
    
    mp_stat = []
    for xth in range(10, 41, 5):
        r = rs[(rs[:,0]==win_size) & (rs[:,1]==xth), 4:]                        
        r = np.sum(r, axis=0)
        mp_stat.append([-1*xth/10, r[0], r[1]])


    r = mp_stat
    r =np.array(r)   
    plt.plot(r[:, 0], r[:, 1], color = 'green', label="Inside")
    plt.plot(r[:, 0], r[:, 2], color = 'red', label="Outside")

    #plt.scatter(r[:, 0], r[:, 1]/1000, color = 'blue')
    #plt.scatter(r[:, 0], r[:, 2]/1000, color = 'green')
    #plt.scatter(r[:, 0], r[:, 3]/1000, color = 'red')

    plt.xlabel('Accelerometer threshold for X axis')
    plt.ylim([500, 1100])
    #plt.xlim([-4, -1])
    plt.ylabel('Detected Bite Count ')
    plt.legend(loc=3)
    plt.title('Window size: {} s'.format(int(win_size/8)))
    plt.grid(True)
    
    plt.show()